In [1]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import pickle

In [2]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Dropout, InputLayer, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import model_from_json
import numpy as np
import pandas as pd

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.math import exp

In [3]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import pickle
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, concatenate, Dropout, Input

In [4]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * exp(-0.1)

In [5]:
from tensorflow.keras.callbacks import Callback

class TerminateOnBaseline(Callback):
    """Callback that terminates training when either acc or val_acc reaches a specified baseline
    """
    def __init__(self, monitor='accuracy', baseline=0.9):
        super(TerminateOnBaseline, self).__init__()
        self.monitor = monitor
        self.baseline = baseline

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        acc = logs.get(self.monitor)
        if acc is not None:
            if acc >= self.baseline:
                print('Epoch %d: Reached baseline, terminating training' % (epoch))
                self.model.stop_training = True

In [12]:
parameters_mlp = { 
    'introduction':  [ 0.2, 200, 8],
    'materials':     [ 0.2, 200, 4],
    'conclusion':  [ 0.2, 200, 4]}

In [32]:
columns = list(range(0, 383))
columns = list(map(str, columns))

section='materials'

folder_to_save = 'models_v1'
path_to_save = "/scratch/cinthiasouza/mv-text-summarizer/notebook/{}".format(folder_to_save)

X_embedd = pd.read_csv("dataset/embed_bert_{}_train.csv".format(section))

y_embedd = X_embedd['label']
X_embedd = X_embedd[columns]

In [33]:
with open('dataset/dataset_{}.pkl'.format('features'), 'rb') as fp:
    dataset = pickle.load(fp)

path_to_save = "/scratch/cinthiasouza/mv-text-summarizer/notebook/models_v6"
class_weight={0:1, 1:1}

section='materials'

#X = dataset[section][0]
#y = dataset[section][2]

model = Sequential()
model.add(Dense(64, input_dim=X.shape[1], activation='relu'))
model.add(Dropout(.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(2, activation='sigmoid'))


model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(
                learning_rate=0.001), metrics=['accuracy'])

one_hot_label = to_categorical(y)

test_size, epochs, batch_size =parameters_mlp.get(section)

X_train, X_valid, y_train, y_valid = train_test_split(X, one_hot_label, stratify=one_hot_label, test_size=test_size)

my_callbacks = [EarlyStopping(patience=20)]

my_callbacks = [EarlyStopping(patience=10, restore_best_weights=True),
                keras.callbacks.LearningRateScheduler(scheduler),
                TerminateOnBaseline(monitor='val_accuracy', baseline=0.9)]
model.fit(
            np.array(X_train), np.array(y_train), validation_data=(X_valid,y_valid), epochs=5,
             batch_size=batch_size, class_weight=class_weight, callbacks=my_callbacks, verbose=True)

Epoch 1/5
12153/12153 [==============================] - 1342s 107ms/step - loss: 0.6972 - accuracy: 0.4951 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 2/5
12153/12153 [==============================] - 1326s 109ms/step - loss: 0.6948 - accuracy: 0.4943 - val_loss: 0.6937 - val_accuracy: 0.5000
Epoch 3/5
12153/12153 [==============================] - 1006s 83ms/step - loss: 0.6943 - accuracy: 0.5065 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/5
12153/12153 [==============================] - 751s 62ms/step - loss: 0.6938 - accuracy: 0.5013 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/5
 4273/12153 [=========>....................] - ETA: 6:53 - loss: 0.6936 - accuracy: 0.5020

KeyboardInterrupt: 

In [ ]:
path_to_save = "/scratch/cinthiasouza/mv-text-summarizer/notebook/models_v1"
model_json = model.to_json()
with open('{}/mlp_embed_{}.json'.format(path_to_save, section), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights('{}/mlp_embed_{}.h5'.format(path_to_save, section))
print("Saved model to disk")

In [24]:
X_test=dataset[section][1]
y_test =one_hot_label = to_categorical(dataset[section][3])

In [26]:
path_to_save = "/scratch/cinthiasouza/mv-text-summarizer/notebook/models_v1"
model_json = model.to_json()
with open('{}/mlp_{}.json'.format(path_to_save, section), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights('{}/mlp_{}.h5'.format(path_to_save, section))
print("Saved model to disk")

Saved model to disk


In [25]:
model.evaluate(X_test, y_test)

792/792 [==============================] - 8s 11ms/step - loss: 0.5262 - accuracy: 0.7972


[0.5262404680252075, 0.7971884608268738]

In [117]:
sum(model.predict_classes(X_test))

873